<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

In [2]:
DBNAME = ''
USER = ''
PASSWORD = ''
HOST = ''
PORT = 5432


In [3]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''select
                    count(id)
                from vacancies
             '''

In [5]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f''' select
                    count(id)
                  from employers
             '''

In [7]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f''' select
                    count(id)
                 from areas
             '''

In [9]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f''' select
                    count(id)
                 from industries
             '''

In [11]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

,count
0,294


In [12]:
# выводы по предварительному анализу данных

- количество вакансий(49197) более чем в два раза превышает количество работодателей, которые расположены в 1362 регионах
 и задействованы в 294 сферах деятельности


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [14]:
# текст запроса
query_4_1 = f''' select
                    a.name area,
                    count(v.id) cnt
                 from vacancies v
                    join areas a on v.area_id = a.id
                group by 1
                order by 2 desc
                limit 5
             '''   

In [15]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


In [16]:
fig = px.bar(
    data_frame=df, #датафрейм
    x="area", #ось x
    y="cnt", #ось y
    color='area', #расцветка в зависимости от страны
    text = 'area', #текст на столбцах
    orientation='v', #ориентация графика
    height=500, #высота
    width=800, #ширина
    title='Количество вакансий в регионах' #заголовок
)
#отображаем его
fig.show()

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [18]:
# текст запроса

query_4_2 = f''' select
                    count(id) cnt
                 from vacancies
                 where salary_from is not null
                    or salary_to   is not null  
             '''   


In [19]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [22]:
# текст запроса
query_4_3 = f''' select
                    round(avg(salary_from)) from_salary_avg,
                    round(avg(salary_to))   to_salary_avg
                from vacancies
             '''
query_4_3_1 = f''' select
                    salary_from,
                    salary_to
                from vacancies
                where (salary_from is not null) or (salary_to is not null)
             '''


In [23]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df_ = pd.read_sql_query(query_4_3_1, connection)
df

,from_salary_avg,to_salary_avg
0,71065.0,110537.0


In [24]:
fig = px.box(
    data_frame = df_,
    x = (['salary_from', 'salary_to']),
    #color=(['salary_from', 'salary_to']),
    height=400,
    width=800,
    title='Распределение зп в вилке от и до',
    points="all"
    
)
fig.update_xaxes( title = 'Разброс в значениях' )
fig.update_yaxes( title = 'ЗП' )
fig.show()

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [25]:
# текст запроса
query_4_4 = f''' select
                    schedule,
                    count(id) cnt
                from vacancies
                group by 1
                order by 2 desc
             '''

In [26]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,schedule,cnt
0,Полный день,36424
1,Удаленная работа,9311
2,Гибкий график,2039
3,Сменный график,1054
4,Вахтовый метод,369


In [27]:
df = df.set_index('schedule')
fig = px.pie(values=df.cnt, names=df.index, title = 'Количество вакансий для разных графиков работы', 
             color= df.index, width=600, height=400)
fig.show()

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [28]:
# текст запроса
query_4_5 = f''' select
                    experience,
                    count(id) cnt
                from vacancies
                group by 1
                order by 2
             '''

In [29]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


In [30]:
df = df.set_index('experience')
fig = px.pie(values=df.cnt, names=df.index, title = 'Количество вакансий в зависимости от опыта', 
             color= df.index, width=600, height=400)
fig.show()

***

In [26]:
# выводы по детальному анализу вакансий

+ по количеству вакансий на первом месте московский регион, где более 5 тыс мест, 
  на втором месте ленинградский регион 2851
+ у более половины вакансий заполнено хотя бы одно поле с желаемой зп.
+ нижнее и верхнее средней зп выше чем медианное значение, что обусловлено выбросами по этим показателям.
+ 74 % желает трудится полный рабочий день.
+ 53.2 имеют опыт от 1 до 3 лет, 2.72 % более 6 лет 

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [31]:
# текст запроса
query_5_1 = f''' select
                    e.name,
                    count(v.id) cnt
                from vacancies v 
                    join employers e on e.id = v.employer_id
                group by 1 
                order by 2 desc
                limit 5
             '''

In [32]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

,name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


In [33]:
fig = px.bar(
    data_frame=df, #датафрейм
    x="name", #ось x
    y="cnt", #ось y
    color='name', #расцветка в зависимости от страны
    text = 'name', #текст на столбцах
    orientation='v', #ориентация графика
    height=500, #высота
    width=800, #ширина
    title='Количество вакансий по работодателям ( 5 top)' #заголовок
)
#отображаем его
fig.show()

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [35]:
# текст запроса
query_5_2 = f'''select  
                    a.name region_name,
                    count(e.id) nbr_emplrs,
                    count(v.id) nbr_vcns
                from areas a
                    left join employers e on e.area = a.id
                    left join vacancies v on a.id = v.area_id
                where v.id is null  
                group by 1
                order by 2 desc
                limit 10
             '''

query_5_2_ = f'''select  
                    a.name region_name,
                    count(e.id) nbrs
                from areas a
                    left join employers e on e.area = a.id
                    left join vacancies v on a.id = v.area_id
                where v.id is not null  
                group by 1
                order by 2 desc
                limit 10
             '''

In [36]:
# результат запроса
df_ = pd.read_sql_query(query_5_2_, connection)
df = pd.read_sql_query(query_5_2, connection)
df

,region_name,nbr_emplrs,nbr_vcns
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
5,Ростовская область,18,0
6,Азербайджан,17,0
7,Нижегородская область,16,0
8,Республика Татарстан,16,0
9,Узбекистан,15,0


In [37]:
fig = px.bar(
    data_frame=df_[0:5], #датафрейм
    x="region_name", #ось x
    y="nbrs", #ось y
    color='region_name', #расцветка в зависимости от страны
    text = 'region_name', #текст на столбцах
    orientation='v', #ориентация графика
    height=500, #высота
    width=800, #ширина
    title='Количество вакансий в регионах' #заголовок
)
#отображаем его
fig.show()

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [38]:
# текст запроса
n = 5
query_5_3 = f'''select
                    e.name employer_name,
                    count(distinct v.area_id) nbrs_region
                from vacancies v
                    left join employers e on v.employer_id = e.id
                group by 1
                order by 2 desc
                limit {n}
             '''

In [39]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

,employer_name,nbrs_region
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


In [40]:
fig = px.bar(
    data_frame=df, #датафрейм
    x="employer_name", #ось x
    y="nbrs_region", #ось y
    color='employer_name', #расцветка в зависимости от страны
    text = 'employer_name', #текст на столбцах
    orientation='v', #ориентация графика
    height=500, #высота
    width=800, #ширина
    title='Количество регионов где публикует вакансии ротодатель ' #заголовок
)
#отображаем его
fig.show()

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [42]:
# текст запроса
query_5_4 = f'''select 
                    'не указана' as sph, count(distinct e.id) nbrs
                from employers e
                    left join employers_industries ei on ei.employer_id = e.id
                    left join industries i on i.id = ei.industry_id
                where i.name is null
                union all
                select 
                    'указана ' , count(distinct e.id)
                from employers e
                    left join employers_industries ei on ei.employer_id = e.id
                    left join industries i on i.id = ei.industry_id
                where i.name is not null
            '''

In [43]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

,sph,nbrs
0,не указана,8419
1,указана,15082


In [44]:
df = df.set_index('sph')
fig = px.pie(values=df.nbrs, names=df.index, title = 'Компании указыващие/не указывающие сферу деятельности', 
             color= df.index, width=600, height=400)
fig.show()

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [45]:
# текст запроса
n=10
query_5_5 = f'''select
                    e.name emplrs_name,
                    count(distinct ei.industry_id) indstrs_nbrs
                from employers e
                    left join employers_industries ei on ei.employer_id = e.id
                    left join industries i on i.id = ei.industry_id
                group by 1
                having count(distinct ei.industry_id) = 4
                order by 1
                 
            '''

In [53]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df[0:5]

,emplrs_name,indstrs_nbrs
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4


In [54]:
# общее кол-во компаний с 4-мя сферами деятельности
len(df)

1132

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [55]:
# текст запроса
query_5_6 = f'''select
                    count(e.id) emplrs_nbrs
                from employers e
                    left join employers_industries ei on ei.employer_id = e.id
                    left join industries i on i.id = ei.industry_id
                where i.name = 'Разработка программного обеспечения'
            '''

In [56]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

,emplrs_nbrs
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [49]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
tabs = pd.read_html(url)
lst=tuple(tabs[1]['Город'])

In [51]:
# текст запроса
query_5_7 = f'''with t1 as (
                select 
                    a.name,
                    count(v.id) cnt
                    from vacancies v
                        left join employers e on v.employer_id = e.id
                        left join areas a on a.id = v.area_id
                    where e.name = 'Яндекс' and 
                    a.name IN {lst}
                    group by 1)
                select * from t1
                union all
                select 'total', sum(t1.cnt) from t1
                order by 2
            '''

In [52]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

In [40]:
# выводы по анализу работодателей


+ в пятерку лидеров по количеству вакансий входят Яндекс,Ростелеком,Тинкофф,Сбер и Газпром нефть. Яндекс примерно 
в пять раз превышает три компании следущих за ним
+ по количеству вакансий бесспорным лидером является Москва, затем следуют Санкт-Петербург, Минск, Алматы.
+ по охвату регионов Яндекс находится на первом месте с показателем 181, с небольшим отрывом идем Ростелеком.
+ 35.8 процентов компаний не указывают свою сферу деятельности.
+ более 1 тыс компаний указывают 4 сферы деятельности, а у 3553 компаний указывают одной из сфер Разработка ПО.
+ у компании Яндекс в городах миллиониках открыто 485 вакансии 

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [117]:
# текст запроса
query_6_1 = f'''select 
                    count(*) from vacancies
                where name ilike '%data%' or name ilike '%данн%'
            '''

In [118]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [120]:
# текст запроса
query_6_2 = f'''select 
                  count(*) 
                from vacancies
                where (name ilike '%data science%'
                  or name ilike '%data scientist%'
                  or name ilike '%machine learning%'
                  or name ilike '%исследователь данных%'
                  or name ilike '%машинн%обучен%'
                  or (name like '%ML%' and name not like '%HTML%'))
                  and ( name ilike '%junior%' 
                  or experience = 'Нет опыта' 
                  or employment = 'Стажировка'
                      )
            '''

In [121]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [123]:
# текст запроса
query_6_3 = f'''select 
                    count(*) 
                from vacancies
                where (name ilike '%data science%'
                    or name ilike '%data scientist%'
                    or name ilike '%machine learning%'
                    or name ilike '%исследователь данных%'
                    or name ilike '%машинн%обучен%'
                    or (name like '%ML%' and name not like '%HTML%'))
                    and 
                        (key_skills ilike '%SQL%' 
                    or key_skills ilike '%postgres%')
            '''

In [124]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [126]:
# текст запроса
query_6_4 = f'''select 
                    count(*) 
                from vacancies
                where (name ilike '%data science%'
                    or name ilike '%data scientist%'
                    or name ilike '%machine learning%'
                    or name ilike '%исследователь данных%'
                    or name ilike '%машинн%обучен%'
                    or (name like '%ML%' and name not like '%HTML%'))
                    and key_skills ilike '%Python%' 
            '''

In [127]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [128]:
# текст запроса

query_6_5 = f'''with t1 as (
    select * 
    from vacancies
    where (name ilike '%data science%'
        or name ilike '%data scientist%'
        or name ilike '%machine learning%'
        or name ilike '%исследователь данных%'
        or name ilike '%машинн%обучен%'
        or (name like '%ML%' and name not like '%HTML%'))
    )
select round(avg(length(key_skills) -length(replace(key_skills, CHR(9), ''))+1), 2) from t1
                '''

In [129]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [130]:
# текст запроса
query_6_6 = f'''
                select experience, round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)),0) avg_ds_salary 
                from vacancies 
                where (name ilike '%data science%'
                    or name ilike '%data scientist%'
                    or name ilike '%machine learning%'
                    or name ilike '%исследователь данных%'
                    or name ilike '%машинн%обучен%'
                    or (name like '%ML%' and name not like '%HTML%'))
                    and (salary_from is not NULL or salary_to is not NULL)    
                group by 1 
            '''

In [131]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

,experience,avg_ds_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

In [53]:
# выводы по предметному анализу

+ всего по профилю исследование данных имеется 1771 вакансии, из них только 51 вакансия для начинающих специалистов
+ 351 вакансии с навыком программирования на Python
+ с опытом от 3 лет и выше запрашивают зп в 3.2 раза больше чем специалист без опыта.

# Общий вывод по проекту

In [132]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

+ всего в базе 49 тыс вакансий
+ по количеству вакансий лидирует москва с большим отрывом и города миллионники
+ ключевые работодатели это крупные ИТ компании как например Яндекс, банки и ресурсодобывающие компании.
+ средняя зарплата от 71 до 110 тыс рублей
+ более 70 процентов вакансий предусматривает полную занятость
+ 53 процента вакансий запрашивают опыт работы от года.
+ в сфере исследований данных 1771 вакансия, требуются люди с опытом от 3 лет. 

### продолжение
+ часть компаний не указывает сферу деятельности, что возможно указывает на проблемы с сбором данных
+ в сфере ит можно разбить на области(названия) сфер деятельности, чтобы более детально посмотреть где больше требуется специалистов.
+ как вариант можно также будет отсортировать по языкам программировния,в дальнейшем чтобы увидеть наиболее востребованные.